# Kaggle 3: Sentence-level Substring Matching vs Regex

In [1]:

import src.models.kaggle_model3 as km3
import src.models.kaggle_model3_regex_inference as km3r
import src.evaluate.model as em
from src.data.kaggle_repository import KaggleRepository

In [2]:
config = {
    "model_path": "../models/kaggle_model3/baseline/params.txt",
}

repo = KaggleRepository()

In [3]:
evaluation = em.evaluate_model(
    repo,
    km3.KaggleModel3(),
    config,
)
evaluation


        Model Evaluation:

        - Run time: 46.5181450843811 seconds, avg: 0.0057693346253728265 seconds per sample
        - True Postive Count: 2446, avg: 0.3033610318739923 per sample
        - Precision: 0.9012527634487841
        - Recall: 0.06857110818311794
        

In [20]:
stats = evaluation.output_statistics
stats

,id,label,statistics
0,7bfd8bb51,dbgap,"{'labels': ['dbgap'], 'stats': ['FN']}"
1,7d3e31302,sass|sass data|school and staffing sass data|s...,"{'labels': ['sass', 'sass data', 'school and s..."
2,3644f959a,foodaps|national household food acquisition an...,"{'labels': ['arms', 'arms data', 'agricultural..."
3,ed3527cf3,database of genotypes and phenotypes|database ...,{'labels': ['database of genotypes and phenoty...
4,236061129,dbgap,"{'labels': ['dbgap'], 'stats': ['FN']}"
...,...,...,...
8058,3cb8b0e09,database of genotypes and phenotypes|database ...,{'labels': ['database of genotypes and phenoty...
8059,324fe1310,dbgap,"{'labels': ['dbgap'], 'stats': ['FN']}"
8060,39e66a274,1000 genomes project|1000 genomes project 1000...,"{'labels': ['1000 genomes project', '1000 geno..."
8061,c411b1b6c,dbgap|american cancer society cancer preventio...,"{'labels': ['dbgap', 'american cancer society ..."


In [6]:
import pandas as pd

merged_df = pd.DataFrame({
    "id": stats.loc[:, ["id"]].values.flatten(),
})

merged_df["label"] = merged_df["id"].apply(
    lambda _id: stats.loc[stats["id"] == _id, "label"].values[0]
)


In [22]:
def merge_stats(row:pd.DataFrame):
    _id = row["id"]
    self_labels = row["statistics_self"]["labels"]
    other_labels = row["statistics_other"]["labels"]
    self_stats = row["statistics_self"]["stats"]
    other_stats = row["statistics_other"]["stats"]

    merged_lbls = []
    merged_stats = []
    for self_lbl in self_labels:
        self_stat = self_stats[self_labels.index(self_lbl)]
        merged_lbls.append(self_lbl)
        
        if self_lbl in other_labels:
            other_stat = other_stats[other_labels.index(self_lbl)]
            if self_stat == "TP" or other_stat == "TP":
                merged_stats.append("TP")
            elif self_stat != other_stat:
                raise ValueError("Inconsistent stats for label: {}. got {} - {}".format(self_lbl, self_stat, other_stat))
            else:
                merged_stats.append(self_stat)
        else:
            merged_stats.append(self_stat)

    for other_lbl in set(other_labels) - set(self_labels):
        other_stat = other_stats[other_labels.index(other_lbl)]
        merged_lbls.append(other_lbl)
        merged_stats.append(other_stat)

    
    return pd.DataFrame({
        "id": [_id],
        "statistics": [dict(labels=merged_lbls, stats=merged_stats)]
    })

stats.loc[:, ["id", "statistics"]].merge(
    stats.loc[:, ["id", "statistics"]],
    on="id",
    suffixes=("_self", "_other"),
).apply(merge_stats)

KeyError: 'id'

In [21]:
stats.loc[:, ["id", "statistics"]]

,id,statistics
0,7bfd8bb51,"{'labels': ['dbgap'], 'stats': ['FN']}"
1,7d3e31302,"{'labels': ['sass', 'sass data', 'school and s..."
2,3644f959a,"{'labels': ['arms', 'arms data', 'agricultural..."
3,ed3527cf3,{'labels': ['database of genotypes and phenoty...
4,236061129,"{'labels': ['dbgap'], 'stats': ['FN']}"
...,...,...
8058,3cb8b0e09,{'labels': ['database of genotypes and phenoty...
8059,324fe1310,"{'labels': ['dbgap'], 'stats': ['FN']}"
8060,39e66a274,"{'labels': ['1000 genomes project', '1000 geno..."
8061,c411b1b6c,"{'labels': ['dbgap', 'american cancer society ..."


In [4]:
evaluation_regex = em.evaluate_model(
    repo,
    km3r.Kaggle3RegexInference(config),
    config,
)
evaluation_regex


        Model Evaluation:

        - Run time: 669.5538220405579 seconds, avg: 0.08304028550670443 seconds per sample
        - True Postive Count: 6977, avg: 0.8653106784075406 per sample
        - Precision: 0.49359745313052705
        - Recall: 0.19591710659328315
        